Semantic Router is a superfast decision-making layer for your LLMs and agents. Rather than waiting for slow LLM generations to make tool-use decisions, we use the magic of semantic vector space to make those decisions — routing our requests using semantic meaning.

https://pypi.org/project/semantic-router/

https://github.com/aurelio-labs/semantic-router/blob/main/docs/03-basic-langchain-agent.ipynb

https://blog.codegpt.co/semantic-router-enhancing-control-in-llm-conversations-68ce905c8d33

the router can identify the topic of the user query and apply the right prompt.

In [1]:
from routes import time_route, supplement_route, business_route, product_route, turn_on_ec2_route, turn_off_ec2_route
from semantic_router import RouteLayer
from semantic_router.encoders import OpenAIEncoder
from dotenv import load_dotenv
load_dotenv(override=True)

routes = [time_route, supplement_route, business_route, product_route, turn_on_ec2_route, turn_off_ec2_route]

c:\Users\ELAFACRB1\Codice\GitHub\langchain-rag-agent-chatbot\.venv\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rl = RouteLayer(encoder=OpenAIEncoder(), routes=routes)

2024-07-25 17:13:41 INFO semantic_router.utils.logger Initializing RouteLayer


In [3]:
choise=rl("should I buy ON whey or MP?")

In [4]:
choise

RouteChoice(name='supplement_brand', function_call=None, similarity_score=None, trigger=None)

In [5]:
choise.name

'supplement_brand'

With Langchain agents

In [6]:
import os
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

llm = ChatGroq(model_name="llama3-70b-8192", groq_api_key=GROQ_API_KEY, temperature=0)

In [7]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from tools.aws import ec2_shutdown_tools, ec2_turnon_tools
from tools.retrieval_eventi import get_relevant_document_tool 
from tools.tavily import web_search_tool 
from tools.utils import get_today_date_tool
tools = [get_relevant_document_tool, get_today_date_tool, web_search_tool, ec2_shutdown_tools, ec2_turnon_tools]

Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


In [8]:
memory1 = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)

In [9]:
agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    max_iterations=2,
    early_stopping_method="generate",
    memory=memory1,
)


c:\Users\ELAFACRB1\Codice\GitHub\langchain-rag-agent-chatbot\.venv\rag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and 

In [11]:
prompt="""Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Web eventi: Utile per cercare informazioni relative a ristoranti, locali o ad eventi.\n> Ottieni data: Useful for getting today\'s date\n> tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me, please output a response in one of two formats:\n\n**Option 1:**\nUse this if you want the human to use a tool.\nMarkdown code snippet formatted in the following schema:\n\n```json\n{{\n    "action": string, \\\\ The action to take. Must be one of Web eventi, Ottieni data, tavily_search_results_json\n    "action_input": string \\\\ The input to the action\n}}\n```\n\n**Option #2:**\nUse this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:\n\n```json\n{{\n    "action": "Final Answer",\n    "action_input": string \\\\ You should put what you want to return to use here\n}}\n```\n\nUSER\'S INPUT\n--------------------\nHere is the user\'s input (remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else"""

In [12]:
print(prompt)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

In [13]:
# update the system prompt
# system_message = """You are a helpful personal trainer working to help users on
# their health and fitness journey. Although you are lovely and helpful, you are
# rather sarcastic and witty. So you must always remember to joke with the user.

# Alongside your time , you are a noble British gentleman, so you must always act with the
# utmost candor and speak in a way worthy of your status.

# Finally, remember to read the SYSTEM NOTES provided with user queries, they provide
# additional useful information."""

# new_prompt = agent.agent.create_prompt(system_message=system_message, tools=[])
# agent.agent.llm_chain.prompt = new_prompt

In [14]:
tools

[Tool(name='Web eventi', description='Utile per cercare informazioni relative a ristoranti, locali o ad eventi.', func=<function get_relevant_document at 0x00000211F9F69DA0>),
 Tool(name='Ottieni data', description="Useful for getting today's date", func=<function get_today_date at 0x00000211F9F6BEC0>),
 TavilySearchResults(),
 Tool(name='Accendi Ec2', description='Utile per accendere un istanza EC2 (VM) sul cloud AWS', func=<function turnon_ec2 at 0x00000211F5C8A160>),
 Tool(name='Spegni Ec2', description='Utile per spegnere un istanza EC2 (VM) sul cloud AWS', func=<function turnoff_ec2 at 0x00000211F5C8A200>)]

In [15]:
def semantic_layer(query: str):
    route = rl(query)
    print(route)
    if route.name == "get_time":
        query += f" (SYSTEM NOTE: use the 'Ottieni data' tool)"
    # elif route.name == "business_inquiry":
    #     query += f" (SYSTEM NOTE: {destinations()})"
    # elif route.name == "product":
    #     query += f" (SYSTEM NOTE: {flights()})"
    elif route.name == "turn_on_ec2":
        query += f" (SYSTEM NOTE: use the 'Accendi Ec2' tool)"
    elif route.name == "turn_off_ec2":
        query += f" (SYSTEM NOTE: use the 'Spegni Ec2' tool)"
    else:
        pass
    return query

In [16]:
query = "Start my VM on EC2"
enriched_query = semantic_layer(query)
enriched_query

name='turn_on_ec2' function_call=None similarity_score=None trigger=None


"Start my VM on EC2 (SYSTEM NOTE: use the 'Accendi Ec2' tool)"

In [18]:
agent(enriched_query)

c:\Users\ELAFACRB1\Codice\GitHub\langchain-rag-agent-chatbot\.venv\rag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'statusCode': 200, 'body': '"EC2 instance started!"'}


{'input': "Start my VM on EC2 (SYSTEM NOTE: use the 'Accendi Ec2' tool)",
 'chat_history': [],
 'output': 'Your VM on EC2 has been started.'}

In [19]:
query="spegni EC2"
agent(semantic_layer(query))

name='turn_off_ec2' function_call=None similarity_score=None trigger=None
{'statusCode': 200, 'body': '"EC2 instance stopped!"'}


{'input': "spegni EC2 (SYSTEM NOTE: use the 'Spegni Ec2' tool)",
 'chat_history': [HumanMessage(content="Start my VM on EC2 (SYSTEM NOTE: use the 'Accendi Ec2' tool)"),
  AIMessage(content='Your VM on EC2 has been started.')],
 'output': 'Your EC2 instance has been stopped.'}